In [52]:
import pandas as pd
import re
list_csv = ['data/kor/attraction_places.csv',
            'data/kor/hotel_places.csv',
            'data/kor/restaurant_places.csv',
            'data/eng/eng_attraction_places.csv',
            'data/eng/eng_hotel_places.csv',
            'data/eng/eng_restaurant_places.csv']
save_csv = ['data/attraction_places.csv',
            'data/hotel_places.csv',
            'data/restaurant_places.csv']
save2_csv = ['data/attraction_dropped.csv',
             'data/hotel_dropped.csv',
             'data/restaurant_dropped.csv']

In [3]:
for csv in list_csv:
    df = pd.read_csv(csv,
                     names=['placeId', 'name_kor', 'location', 'class'],
                     encoding='euc-kr')
    print(len(df))

561
804
23150
564
899
12599


In [54]:
for i in range(3):
    df_kor = pd.read_csv(list_csv[i],
                     names=['placeId', 'name_kor', 'location', 'class'],
                     encoding='euc-kr')
    tmp_class = df_kor['class'][0]
    df_eng = pd.read_csv(list_csv[i+3],
                     names=['placeId', 'name_eng', 'location', 'class'],
                     encoding='euc-kr')
    df_eng = df_eng.iloc[:, [j for j, c in enumerate(df_eng.columns) if j != 4]]
    df_eng = df_eng.drop(columns=['location', 'class'])
    df_kor = pd.merge(df_kor, df_eng, how='outer')
    cols = df_kor.columns.tolist()
    cols = cols[3:4] + cols[0:2] + cols[-1:] + cols[2:3]
    df_kor = df_kor[cols]
    df_kor['class'] = df_kor['class'].fillna(tmp_class)
    # 테스트용 좌표 외 drop
    df_dropped = df_kor.dropna(subset=['location'])
    series_match = df_dropped['location'].apply(lambda x:
                                            re.fullmatch('[0-9]+\.[0-9]+,[0-9]+\.[0-9]+', x))
    df_dropped = df_dropped[pd.notnull(series_match)]
    ###
    df_kor.to_csv(save_csv[i], index=False)
    df_dropped.to_csv(save2_csv[i], index=False)
    print(len(df_kor))
    print(len(df_dropped))

569
352
1163
687
23171
18250


In [51]:
print('전체',len(df_kor))
print('NA',len(df_kor[pd.isna(df_kor['location'])]))
df_tmp = df_kor.dropna(subset=['location'])
series_match = df_tmp['location'].apply(lambda x:
                                        re.fullmatch('[0-9]+\.[0-9]+,[0-9]+\.[0-9]+', x))
print('좌표',len(df_tmp[pd.notnull(series_match)]))
print('주소',len(df_tmp[pd.isnull(series_match)]))

전체 23171
NA 26
좌표 18250
주소 4895
